# RFM Analysis - Olist E-Commerce Dataset (Brazil)

**Dataset**: Brazilian E-Commerce Public Dataset by Olist

**Source**: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

**Description**: Real anonymized Brazilian e-commerce data (2016-2018) with multiple tables

**Complexity**: High

## Dataset Structure
Multiple CSVs:
- `olist_orders_dataset.csv` - Order information
- `olist_order_items_dataset.csv` - Items within orders
- `olist_customers_dataset.csv` - Customer information
- `olist_order_payments_dataset.csv` - Payment details
- `olist_order_reviews_dataset.csv` - Customer reviews
- `olist_products_dataset.csv` - Product information
- `olist_sellers_dataset.csv` - Seller information
- `olist_geolocation_dataset.csv` - Geographic data

## Focus Areas
- Multi-table data integration
- RFM with product categories
- Geographic segmentation
- Review sentiment impact on RFM

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Load All Datasets

In [ ]:
# Load all tables
# Download from: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

orders = pd.read_csv('olist_orders_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
customers = pd.read_csv('olist_customers_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')

print("Dataset Shapes:")
print(f"Orders: {orders.shape}")
print(f"Order Items: {order_items.shape}")
print(f"Customers: {customers.shape}")
print(f"Payments: {payments.shape}")
print(f"Reviews: {reviews.shape}")
print(f"Products: {products.shape}")

In [ ]:
# Explore orders table
print("Orders columns:")
print(orders.columns.tolist())
print("\nOrder status distribution:")
print(orders['order_status'].value_counts())
print("\nDate range:")
print(f"From {orders['order_purchase_timestamp'].min()} to {orders['order_purchase_timestamp'].max()}")

## 2. Data Integration and Preprocessing

In [ ]:
# Convert date columns
date_columns = ['order_purchase_timestamp', 'order_approved_at', 
                'order_delivered_carrier_date', 'order_delivered_customer_date', 
                'order_estimated_delivery_date']

for col in date_columns:
    if col in orders.columns:
        orders[col] = pd.to_datetime(orders[col])

# Filter only delivered orders for RFM
orders_delivered = orders[orders['order_status'] == 'delivered'].copy()
print(f"Delivered orders: {len(orders_delivered):,}")
print(f"Unique customers: {orders_delivered['customer_id'].nunique():,}")

In [ ]:
# Merge datasets to create comprehensive transaction table
# Orders + Customers
df = orders_delivered.merge(customers, on='customer_id', how='left')

# Add order items with product info
order_items_enhanced = order_items.merge(products[['product_id', 'product_category_name']], 
                                         on='product_id', how='left')

# Aggregate order items to order level
order_summary = order_items_enhanced.groupby('order_id').agg({
    'price': 'sum',
    'freight_value': 'sum',
    'order_item_id': 'count',  # number of items
    'product_category_name': lambda x: ', '.join(x.dropna().unique())  # categories
}).reset_index()

order_summary.columns = ['order_id', 'items_price', 'freight_value', 'num_items', 'categories']
order_summary['order_value'] = order_summary['items_price'] + order_summary['freight_value']

# Merge with main dataframe
df = df.merge(order_summary, on='order_id', how='left')

# Add payment info
payment_summary = payments.groupby('order_id').agg({
    'payment_value': 'sum',
    'payment_type': lambda x: x.mode()[0] if len(x.mode()) > 0 else None
}).reset_index()

df = df.merge(payment_summary, on='order_id', how='left')

# Add review scores
review_summary = reviews[['order_id', 'review_score']]
df = df.merge(review_summary, on='order_id', how='left')

print(f"\nIntegrated dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

In [ ]:
# Data quality check
print("Missing values:")
print(df.isnull().sum())

# Use payment_value as final order amount (most reliable)
df['final_amount'] = df['payment_value'].fillna(df['order_value'])

# Remove orders with invalid amounts
df = df[df['final_amount'] > 0]

print(f"\nFinal dataset for RFM: {len(df):,} orders from {df['customer_id'].nunique():,} customers")

## 3. Basic RFM Calculation

In [ ]:
# Set analysis date
analysis_date = df['order_purchase_timestamp'].max() + timedelta(days=1)
print(f"Analysis date: {analysis_date}")

# Calculate RFM
rfm = df.groupby('customer_id').agg({
    'order_purchase_timestamp': lambda x: (analysis_date - x.max()).days,
    'order_id': 'nunique',
    'final_amount': 'sum'
}).reset_index()

rfm.columns = ['customer_id', 'Recency', 'Frequency', 'Monetary']

print("RFM Summary:")
print(rfm.describe())

## 4. Enhanced RFM with Additional Features

In [ ]:
# Add geographic information
customer_location = df.groupby('customer_id')['customer_city'].first()
customer_state = df.groupby('customer_id')['customer_state'].first()

rfm['city'] = rfm['customer_id'].map(customer_location)
rfm['state'] = rfm['customer_id'].map(customer_state)

# Add average review score
avg_review = df.groupby('customer_id')['review_score'].mean()
rfm['avg_review_score'] = rfm['customer_id'].map(avg_review)

# Add product diversity (number of unique categories purchased)
def count_categories(customer_id):
    customer_orders = df[df['customer_id'] == customer_id]['categories'].str.split(', ')
    all_categories = []
    for cats in customer_orders.dropna():
        all_categories.extend(cats)
    return len(set(all_categories))

# For performance, calculate for sample or use vectorized approach
category_diversity = df.groupby('customer_id')['categories'].apply(
    lambda x: len(set(','.join(x.dropna()).split(', '))) if len(x.dropna()) > 0 else 0
)
rfm['category_diversity'] = rfm['customer_id'].map(category_diversity)

# Average order value
rfm['avg_order_value'] = rfm['Monetary'] / rfm['Frequency']

# Payment method preference
payment_pref = df.groupby('customer_id')['payment_type'].apply(lambda x: x.mode()[0] if len(x.mode()) > 0 else None)
rfm['preferred_payment'] = rfm['customer_id'].map(payment_pref)

print("Enhanced RFM features:")
print(rfm.head())

## 5. RFM Scoring and Segmentation

In [ ]:
# Standard RFM scoring
rfm['R_Score'] = pd.qcut(rfm['Recency'], q=5, labels=[5, 4, 3, 2, 1], duplicates='drop').astype(int)
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5], duplicates='drop').astype(int)
rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=5, labels=[1, 2, 3, 4, 5], duplicates='drop').astype(int)

rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Total'] = rfm['R_Score'] + rfm['F_Score'] + rfm['M_Score']

# Segmentation
def segment_customers(df):
    segments = []
    for _, row in df.iterrows():
        r, f, m = row['R_Score'], row['F_Score'], row['M_Score']
        
        if r >= 4 and f >= 4 and m >= 4:
            segment = 'Champions'
        elif r >= 3 and f >= 4:
            segment = 'Loyal Customers'
        elif r >= 4 and 2 <= f <= 3:
            segment = 'Potential Loyalists'
        elif r >= 4 and f <= 2:
            segment = 'New Customers'
        elif 3 <= r <= 4 and f <= 2:
            segment = 'Promising'
        elif r >= 3 and f >= 3 and m >= 3:
            segment = 'Need Attention'
        elif 2 <= r <= 3:
            segment = 'About to Sleep'
        elif r <= 2 and f >= 4 and m >= 4:
            segment = 'At Risk'
        elif r <= 1 and f >= 4 and m >= 4:
            segment = "Can't Lose Them"
        elif r <= 2 and f <= 2:
            segment = 'Hibernating'
        else:
            segment = 'Lost'
        segments.append(segment)
    return segments

rfm['Segment'] = segment_customers(rfm)

print("Segment Distribution:")
print(rfm['Segment'].value_counts())

## 6. Geographic Analysis

In [ ]:
# RFM by state
state_summary = rfm.groupby('state').agg({
    'customer_id': 'count',
    'Monetary': ['sum', 'mean'],
    'Frequency': 'mean',
    'Recency': 'mean'
}).round(2)

state_summary.columns = ['_'.join(col).strip() for col in state_summary.columns.values]
state_summary = state_summary.rename(columns={'customer_id_count': 'num_customers'})
state_summary = state_summary.sort_values('Monetary_sum', ascending=False)

print("Top 10 States by Revenue:")
print(state_summary.head(10))

In [ ]:
# Visualize geographic distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Revenue by state
top_states = state_summary.head(10)
top_states['Monetary_sum'].plot(kind='barh', ax=axes[0], color='darkblue')
axes[0].set_xlabel('Total Revenue (R$)', fontsize=12)
axes[0].set_title('Top 10 States by Revenue', fontsize=14, fontweight='bold')

# Customer count by state
top_states['num_customers'].plot(kind='barh', ax=axes[1], color='darkgreen')
axes[1].set_xlabel('Number of Customers', fontsize=12)
axes[1].set_title('Top 10 States by Customer Count', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Segment distribution by top states
top_5_states = state_summary.head(5).index
state_segment = rfm[rfm['state'].isin(top_5_states)].groupby(['state', 'Segment']).size().unstack(fill_value=0)

state_segment.plot(kind='bar', stacked=True, figsize=(14, 6), colormap='tab10')
plt.title('Customer Segments by Top 5 States', fontsize=14, fontweight='bold')
plt.xlabel('State', fontsize=12)
plt.ylabel('Number of Customers', fontsize=12)
plt.legend(title='Segment', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 7. Review Score Analysis

In [ ]:
# Review score distribution by segment
review_by_segment = rfm.groupby('Segment')['avg_review_score'].agg(['mean', 'median', 'count']).round(2)
review_by_segment = review_by_segment.sort_values('mean', ascending=False)

print("Average Review Score by Segment:")
print(review_by_segment)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

review_by_segment['mean'].plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_xlabel('Average Review Score', fontsize=12)
axes[0].set_title('Average Review Score by Segment', fontsize=14, fontweight='bold')
axes[0].axvline(rfm['avg_review_score'].mean(), color='red', linestyle='--', 
                label=f'Overall Avg: {rfm["avg_review_score"].mean():.2f}')
axes[0].legend()

# Review score distribution
rfm['avg_review_score'].hist(bins=20, ax=axes[1], color='coral', edgecolor='black')
axes[1].set_xlabel('Average Review Score', fontsize=12)
axes[1].set_ylabel('Number of Customers', fontsize=12)
axes[1].set_title('Customer Review Score Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation between review score and RFM metrics
correlation_data = rfm[['Recency', 'Frequency', 'Monetary', 'avg_review_score', 'category_diversity']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_data, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={'label': 'Correlation'})
plt.title('Correlation Matrix: RFM, Reviews, and Product Diversity', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 8. Product Category Analysis

In [ ]:
# Category diversity by segment
category_diversity_segment = rfm.groupby('Segment')['category_diversity'].agg(['mean', 'median']).round(2)
category_diversity_segment = category_diversity_segment.sort_values('mean', ascending=False)

print("Average Category Diversity by Segment:")
print(category_diversity_segment)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

category_diversity_segment['mean'].plot(kind='barh', ax=axes[0], color='darkgreen')
axes[0].set_xlabel('Average Number of Categories', fontsize=12)
axes[0].set_title('Product Category Diversity by Segment', fontsize=14, fontweight='bold')

# Scatter: Category diversity vs Monetary
axes[1].scatter(rfm['category_diversity'], rfm['Monetary'], alpha=0.5, c='steelblue')
axes[1].set_xlabel('Category Diversity', fontsize=12)
axes[1].set_ylabel('Monetary Value (R$)', fontsize=12)
axes[1].set_title('Category Diversity vs Customer Value', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 9. Payment Method Analysis

In [ ]:
# Payment method by segment
payment_segment = pd.crosstab(rfm['Segment'], rfm['preferred_payment'], normalize='index') * 100

payment_segment.plot(kind='bar', stacked=True, figsize=(14, 6), colormap='Set3')
plt.title('Payment Method Preference by Segment (%)', fontsize=14, fontweight='bold')
plt.xlabel('Segment', fontsize=12)
plt.ylabel('Percentage', fontsize=12)
plt.legend(title='Payment Method', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nPayment Method Distribution by Segment:")
print(payment_segment.round(2))

## 10. Advanced Visualizations

In [ ]:
# Multi-dimensional analysis
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

# Color by review score
colors = rfm['avg_review_score'].fillna(rfm['avg_review_score'].mean())
scatter = ax.scatter(rfm['Recency'], rfm['Frequency'], rfm['Monetary'], 
                     c=colors, cmap='RdYlGn', alpha=0.6, s=50)

ax.set_xlabel('Recency (days)', fontsize=11)
ax.set_ylabel('Frequency', fontsize=11)
ax.set_zlabel('Monetary (R$)', fontsize=11)
ax.set_title('3D RFM Analysis (Colored by Review Score)', fontsize=14, fontweight='bold')

cbar = plt.colorbar(scatter, ax=ax, pad=0.1)
cbar.set_label('Avg Review Score', fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
# Segment profile heatmap
segment_profile = rfm.groupby('Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'avg_review_score': 'mean',
    'category_diversity': 'mean',
    'avg_order_value': 'mean'
})

# Normalize for better visualization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
segment_profile_scaled = pd.DataFrame(
    scaler.fit_transform(segment_profile),
    index=segment_profile.index,
    columns=segment_profile.columns
)

plt.figure(figsize=(12, 8))
sns.heatmap(segment_profile_scaled.T, annot=True, fmt='.2f', cmap='RdYlGn', 
            center=0, linewidths=1, cbar_kws={'label': 'Standardized Value'})
plt.title('Customer Segment Profile (Standardized)', fontsize=14, fontweight='bold')
plt.xlabel('Segment', fontsize=12)
plt.ylabel('Metric', fontsize=12)
plt.tight_layout()
plt.show()

## 11. Business Insights and Recommendations

In [ ]:
# Comprehensive business metrics
total_customers = len(rfm)
total_revenue = rfm['Monetary'].sum()
avg_customer_value = rfm['Monetary'].mean()

print("=" * 70)
print("OLIST BRAZIL E-COMMERCE - RFM ANALYSIS INSIGHTS")
print("=" * 70)

print(f"\n1. OVERALL METRICS")
print(f"   Total Customers: {total_customers:,}")
print(f"   Total Revenue: R$ {total_revenue:,.2f}")
print(f"   Average Customer Value: R$ {avg_customer_value:,.2f}")
print(f"   Average Review Score: {rfm['avg_review_score'].mean():.2f}/5.0")

print(f"\n2. GEOGRAPHIC INSIGHTS")
top_3_states = state_summary.head(3)
for idx, (state, row) in enumerate(top_3_states.iterrows(), 1):
    print(f"   #{idx} {state}: {row['num_customers']:,.0f} customers, R$ {row['Monetary_sum']:,.2f} revenue")
print(f"   → ACTION: Focus marketing and logistics in top states")

print(f"\n3. SEGMENT BREAKDOWN")
for segment in rfm['Segment'].unique():
    seg_data = rfm[rfm['Segment'] == segment]
    seg_count = len(seg_data)
    seg_revenue = seg_data['Monetary'].sum()
    seg_review = seg_data['avg_review_score'].mean()
    print(f"\n   {segment}:")
    print(f"   - Count: {seg_count:,} ({seg_count/total_customers*100:.1f}%)")
    print(f"   - Revenue: R$ {seg_revenue:,.2f} ({seg_revenue/total_revenue*100:.1f}%)")
    print(f"   - Avg Review: {seg_review:.2f}/5.0")

print(f"\n4. CUSTOMER SATISFACTION")
high_satisfaction = len(rfm[rfm['avg_review_score'] >= 4])
low_satisfaction = len(rfm[rfm['avg_review_score'] < 3])
print(f"   High Satisfaction (≥4): {high_satisfaction:,} ({high_satisfaction/total_customers*100:.1f}%)")
print(f"   Low Satisfaction (<3): {low_satisfaction:,} ({low_satisfaction/total_customers*100:.1f}%)")
print(f"   → ACTION: Investigate low-satisfaction customers for retention")

print(f"\n5. PRODUCT STRATEGY")
avg_diversity = rfm['category_diversity'].mean()
high_diversity = len(rfm[rfm['category_diversity'] >= 3])
print(f"   Average Categories per Customer: {avg_diversity:.2f}")
print(f"   Customers with 3+ categories: {high_diversity:,} ({high_diversity/total_customers*100:.1f}%)")
print(f"   → ACTION: Cross-sell campaigns to increase category diversity")

print(f"\n6. PAYMENT INSIGHTS")
payment_dist = rfm['preferred_payment'].value_counts()
print(f"   Most popular: {payment_dist.index[0]} ({payment_dist.iloc[0]/len(rfm)*100:.1f}%)")
print(f"   → ACTION: Optimize payment options based on preferences")

print("\n" + "=" * 70)

## 12. Export Results

In [ ]:
# Export comprehensive RFM results
rfm.to_csv('olist_brazil_rfm_analysis.csv', index=False)
print("RFM analysis exported to: olist_brazil_rfm_analysis.csv")

# Export segment summaries
segment_summary = rfm.groupby('Segment').agg({
    'customer_id': 'count',
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'sum'],
    'avg_review_score': 'mean',
    'category_diversity': 'mean'
}).round(2)

segment_summary.to_csv('olist_segment_summary.csv')
print("Segment summary exported to: olist_segment_summary.csv")

# Export geographic analysis
state_summary.to_csv('olist_state_analysis.csv')
print("State analysis exported to: olist_state_analysis.csv")